# Image Viewer

The current version of the image viewer allows for efficient lookup of any of the thousands of DICOM images in the DSB 2017 dataset.  The image viewer does not yet support RAW files

**Improvements**
* Add support for RAW files (file format of LUNA images)
* Incorporate whether or not patient has cancer into the display
* For LUNA images, highlight labeled nodes with circle

In [ ]:
import os
import dicom
import numpy as np
from ipywidgets import widgets, interact, interactive, IntSlider
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load list of all patients
# scan_path = os.path.join(os.getcwd(),'stage1') #Assume this notebook is in the same folder as the data
scan_path = 'E:\\DSB 2017\\stage1\\stage1' #Hard code the location of the data
patients = sorted(os.listdir(scan_path))

In [ ]:
#Instantiate Controls
patient_list = widgets.Select(description = 'Patients',options = patients)  # populate the drop down with the sorted list of patients
img_slider = IntSlider(description = "Image Number", min = 0) # create image slider and set min = 0; we won't know the max until a patient is selected

In [ ]:
# Function to load a pations data
def load_patient(args = ''):
    patient_path = os.path.join(scan_path,patient_list.value)  #store the path to the selected patient
    global slices
    slices = [dicom.read_file(patient_path + '/' + s) for s in os.listdir(patient_path)] #read in the individual slices
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2])) #sort slices based on vertical position
    img_slider.max = len(slices) - 1 #set the max of the image slider
    plot_img()

In [ ]:
def plot_img(args = ''):
    img = slices[img_slider.value].pixel_array #extract the image data from the selected slice
    clear_output()
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(img,cmap=plt.cm.gray)
    ax.axes.axis('off')

In [ ]:
patient_list.observe(load_patient,'value')
img_slider.observe(plot_img,'value')

display(patient_list)
display(img_slider)

load_patient()